<center>

<h1 style="text-align:center"> Polymorphic Lambda Calculus: System F </h1>
<h2 style="text-align:center"> CS3100 Fall 2019 </h2>
</center>

## Review

### Previously

* Simply Typed Lambda Calulus.
  + Products, Sums, Type Soundness
  + Curry Howard Correspondence
  + Type Erasure

### Today

* System F: Polymorphic Lambda Calculus

$
\newcommand{\stlc}{\lambda^{\rightarrow}}
\require{color}
\newcommand{\c}[2]{{\color{#1}{\text{#2}}}}
$

## Doubling functions

In simply typed lambda calculus, the `twice f x = f (f x)` function must be specified at every type:

$
twice\_unit = \lambda f:1 \rightarrow 1.\lambda x:1. f~(f ~x) \\
twice\_sum = \lambda f: A+B \rightarrow A+B.\lambda x:A+B. f~(f ~x) \\
twice\_a2a = \lambda f: (A \rightarrow A) \rightarrow (A \rightarrow A). \lambda x:A \rightarrow A. f ~(f ~x)
$

Clearly this copy-pasting of code and changing the type violates the basic dictum of software engineering:

**Abstraction Principle:** Each significant piece of functionality in a program should be implemented in just one place in the source code.

## Universal Type

* System F (polymorphic lambda calculus) is obtained by extending Curry Howard isomorphism to $\forall$.
* System F has dedicated syntax for type and term families such as `twice`:
  + The System F term $\Lambda \alpha.\lambda f: \alpha \rightarrow \alpha.\lambda x:\alpha.f ~(f ~x))$ has type $\forall \alpha.(\alpha \rightarrow \alpha) \rightarrow \alpha \rightarrow \alpha$.
* $\Lambda \alpha.M$ is called a **type abstraction**.
  + Correspondingly, we also have a **type application** of the form $M A$, where $M$ is a term and $A$ is a type.
* System F offers polymorphism
  + $\c{red}{But which kind?}$

## Polymorphism

Polymorphism comes in various kinds

1. Ad-hoc polymorphism: 
   * function overloading (C++,Java)
   * operator overloading (C++,Java,Standard ML, C)
   * typeclasses (Haskell)
2. Subtyping:
   * subclasses (C++, Java, OCaml)
   * Row polymorphism (OCaml)
3. Parametric Polymorphism:
   * polymorphic data and functions in OCaml, Haskell, Standard ML.
   * Generics in Java, C#

## Polymorphism

* Unqualified term "polymorphism" means different things depending on who you talk to
  + OO person: subtype polymorphism; parametric polymorphism is generics
  + FP person: parametric polymorphism.

## Types

Types in System F are as follows:

\\[
\begin{array}{rcll}
\text{Types: } A,B & ::=  & \alpha & \text{(type variable)} \\
                   & \mid & A \rightarrow B & \text{(function type)} \\
                   & \mid & \forall \alpha.A & \text{(universal type)}
\end{array}
\\]

* We have dropped pairs, sums, 1 (unit), 0 types from $\lambda^{\rightarrow}$
  + Can be encoded!
  + Recall that $\lambda^{\rightarrow}$ without base types was degenerate.
* Notice the symmetry to untyped lambda calculus terms.

## Free Type Variables

We define free type variables on System F types similar to free variables on lambda terms.

\\[
\begin{array}{rcl}
FTV(\alpha) & = & \alpha \\
FTV(A \rightarrow B) & = & FTV(A) \cup FTV(B) \\
FTV(\forall \alpha.A) & = & FTV(A) \setminus \{\alpha\}
\end{array}
\\]

$\newcommand{\inferrulel}[3]{\displaystyle{\frac{#1}{#2}~~{\small #3}}}$

## Typing Rules

\\[
\begin{array}{cc}
\inferrulel{}{\Gamma,x:A \vdash x:A}{(var)} \\ \\
\inferrulel{\Gamma \vdash M : A \rightarrow B \quad \Gamma \vdash N : A}{\Gamma \vdash M~N : B}{(\rightarrow elim)} &
\inferrulel{\Gamma,x:A \vdash M : B}{\Gamma \vdash \lambda x:A.M : A \rightarrow B}{(\rightarrow intro)} \\ \\
\inferrulel{\Gamma \vdash M : \forall \alpha.A}
           {\Gamma \vdash M ~B : A[B/\alpha]}{(\forall elim)} &
\inferrulel{\Gamma \vdash M : A \quad \alpha \notin FTV(\Gamma)}
           {\Gamma \vdash \Lambda \alpha.M : \forall \alpha.A}
           {(\forall intro)}
\end{array}
\\]

## Terms

\\[
\begin{array}{rcll}
\text{Terms: } M,N & ::=  & x & \text{(variable)} \\
                   & \mid & M~N & \text{(application)} \\
                   & \mid & M~[A] & \text{(type application)} \\
                   & \mid & \lambda x:A.M & \text{(abstraction)} \\
                   & \mid & \Lambda \alpha.M & \text{(type abstraction)}
\end{array}
\\]

$\newcommand{\inferrule}[2]{\displaystyle{\frac{#1}{#2}}}$

## Substitution

### For terms

If $M$ is a term, $N$ a term, and $x$ a variable, we write $M[N/x]$ for capture-free substitution of $N$ for $x$ in $M$.

### For types

If $M$ is a term, $B$ a type, and $\alpha$ a type variable, we write $M[B/\alpha]$ for capture-free substitution of $B$ for $\alpha$ in $M$.


## Reduction: Boring Rules

\\[
\begin{array}{ccc}
\inferrule{M \rightarrow M'}{M~N \rightarrow M'~N} & 
\inferrule{N \rightarrow N'}{M~N \rightarrow M~N'} &
\inferrule{M \rightarrow M'}{\lambda x.M \rightarrow \lambda x.M'}
\end{array}
\\]

\\[
\begin{array}{cc}
\inferrule{M \rightarrow M'}{M~A \rightarrow M'~A} &
\inferrule{M \rightarrow M'}{\Lambda \alpha.M \rightarrow \Lambda \alpha.M'}
\end{array}
\\]

## Reductions: Interesting Rules

\\[
\begin{array}{rcl}
(\lambda x:A.M) ~N       & \rightarrow & M[N/x] \\
(\Lambda \alpha.M) ~A    & \rightarrow & M[A/\alpha] \\
\lambda x:A.M ~x         & \rightarrow & \lambda M \quad \text{, if } x \notin FV(M) \\
\lambda \alpha.M ~\alpha & \rightarrow & \lambda M \quad \text{, if } \alpha \notin FTV(M)
\end{array}
\\]

## Encodings : Booleans

In untyped lambda calculus, `tru` and `fls` values were:

```ocaml
tru = 𝜆t.𝜆f.t
fls = 𝜆t.𝜆f.f
```

In simply typed lambda calculus, there was a `tru` and `fls` for each type:

```ocaml
tru_int = 𝜆t:int.𝜆f:int.t
tru_bool = 𝜆t:float.𝜆f:float.t
```

## Encoding Booleans

In System F, there is a single polymorphic `tru` and `fls` values of type `bool`. We define the type `bool` and the booleans as

\\[
\begin{array}{rcl}
bool & = & \forall \alpha. \alpha \rightarrow \alpha \rightarrow \alpha \\
tru : bool & = & \Lambda \alpha. \lambda t:\alpha. \lambda f:\alpha. t \\
fls : bool & = & \Lambda \alpha. \lambda t:\alpha. \lambda f:\alpha. f \\
\end{array}
\\]

Easy to see that the judgements $\vdash tru : bool$ and $\vdash fls : bool$ hold.

## Encoding boolean operations

`test` function is defined as:

\\[
\begin{array}{rcl}
test : \forall \alpha. bool \rightarrow \alpha \rightarrow \alpha \rightarrow \alpha \\
test = \Lambda \alpha. \lambda b : bool. b~\alpha
\end{array}
\\]

Notice the **type application** of $b$ to $\alpha$ above. $test$ at $bool$ type is:

\\[
test\_bool : bool \rightarrow bool \rightarrow bool = test ~bool
\\]

We can define logical operators as follows:

\\[
\begin{array}{rcl}
and : bool \rightarrow bool \rightarrow bool & = & \lambda x:bool.\lambda y:bool.x ~bool ~y ~fls \\
or  : bool \rightarrow bool \rightarrow bool& = & \lambda x:bool.\lambda y:bool.x ~bool ~tru ~y \\
not : bool \rightarrow bool & = & \lambda x:bool.x ~bool ~fls ~tru 
\end{array}
\\]

## Encoding natural numbers

The type for `nat` is defined as

\\[
nat = \forall \alpha. (\alpha \rightarrow \alpha) \rightarrow \alpha \rightarrow \alpha
\\]

Now we can define church numerals as

\\[
\begin{array}{rcl}
zero : nat & = & \Lambda \alpha. \lambda s : \alpha \rightarrow \alpha. \lambda z : \alpha. z \\
one  : nat & = & \Lambda \alpha. \lambda s : \alpha \rightarrow \alpha. \lambda z : \alpha. s~z \\
two  : nat & = & \Lambda \alpha. \lambda s : \alpha \rightarrow \alpha. \lambda z : \alpha. s~(s~z)
\end{array}
\\]

## Pairs

* Unlike $\lambda^{\rightarrow}$, we did not include a pair type in our types.

\\[
\begin{array}{rcll}
\text{Types: } A,B & ::=  & \alpha & \text{(type variable)} \\
                   & \mid & A \rightarrow B & \text{(function type)} \\
                   & \mid & \forall \alpha.A & \text{(universal type)}
\end{array}
\\]

* We can encode the pair **type**!
  + We encoded pair **terms** in untyped lambda calculus.
  + System F allows encoding of **types** as well as **terms**.

## Pairs

Pair type is 

\\[
A \times B = \forall \alpha.(A \rightarrow B \rightarrow \alpha) \rightarrow \alpha
\\]

(Why is the pair type this?)

Given $M : A$ and $N : B$, the pair is

\\[
\langle M,N \rangle : A \times B = \Lambda \alpha.\lambda f:(A \rightarrow B \rightarrow \alpha). f ~M ~N
\\]

The projection functions are

\\[
\begin{array}{c}
fst = \Lambda \alpha. \Lambda \beta. \lambda p : \alpha \times \beta. p~\alpha~(\lambda x:\alpha.\lambda y:\beta.x) \\
snd = \Lambda \alpha. \Lambda \beta. \lambda p : \alpha \times \beta. p~\beta~(\lambda x:\alpha.\lambda y:\beta.y)
\end{array}
\\]

## Curry Howard Correspondence

\\[
\begin{array}{cc}
\inferrulel{}{\Gamma,x:A \vdash A}{(var)} \\ \\
\inferrulel{\Gamma \vdash A \implies B \quad \Gamma \vdash A}
           {\Gamma \vdash B}{(\implies elim)} &
\inferrulel{\Gamma,x:A \vdash B}
           {\Gamma \vdash A \implies B}
           {(\implies intro)} \\ \\
\inferrulel{\Gamma \vdash \forall \alpha.A}
           {\Gamma \vdash A[B/\alpha]}{(\forall ~elim)} &
\inferrulel{\Gamma \vdash A \quad \alpha \notin FTV(\Gamma)}
           {\Gamma \vdash \Lambda \alpha.M : \forall \alpha.A}
           {(\forall ~intro)}
\end{array}
\\]

## Curry Howard Correspondence

\\[
\begin{array}{cc}
\inferrulel{\Gamma \vdash \forall \alpha.A}
           {\Gamma \vdash A[B/\alpha]}{(\forall ~elim)} &
\inferrulel{\Gamma \vdash A \quad \alpha \notin FTV(\Gamma)}
           {\Gamma \vdash \Lambda \alpha.M : \forall \alpha.A}
           {(\forall ~intro)}
\end{array}
\\]

* $\forall~intro$ is universal generalization
  + If a statement has been proved for arbitrary $\alpha$ then it holds for every $\alpha$.
* $a \wedge b \implies a$
  * $\neg (a \wedge b) \vee a$
  * $\neg a \vee \neg b \vee a$
  * $\top \vee \neg b$
  * $\top$
  * Hence, $\forall a,b.a \wedge b \implies a$
* Capture arbitrary $\alpha$ by saying that $\alpha$ is not an assumption in $\Gamma$ i.e) not a free type variable for $\Gamma$.

## Curry Howard Correspondence

\\[
\begin{array}{cc}
\inferrulel{\Gamma \vdash \forall \alpha.A}
           {\Gamma \vdash A[B/\alpha]}{(\forall ~elim)} &
\inferrulel{\Gamma \vdash A \quad \alpha \notin FTV(\Gamma)}
           {\Gamma \vdash \Lambda \alpha.M : \forall \alpha.A}
           {(\forall ~intro)}
\end{array}
\\]

* $\forall~elim$ is universal specialisation
  + If a statement is true for all propositions $\alpha$ it also holds for some proposition $B$.

## Missing Logical Connectives

It turns out that $\forall$ and $\implies$ is sufficient to encode other logical connectives.

\\[
\begin{array}{rcl}
A \wedge B & \Leftrightarrow & \forall \alpha. (A \Rightarrow B \Rightarrow \alpha) \Rightarrow \alpha \\
A \vee B & \Leftrightarrow & \forall \alpha. (A \Rightarrow \alpha) \Rightarrow (B \Rightarrow \alpha) \Rightarrow \alpha \\
\neg A & \Leftrightarrow & \forall \alpha.A \Rightarrow \alpha \\
\top & \Leftrightarrow & \forall \alpha.\alpha \Rightarrow \alpha \\
\bot & \Leftrightarrow & \forall \alpha.\alpha \\
\exists \beta.A & \Leftrightarrow & \forall \alpha.(\forall \beta.A \Rightarrow \alpha) \Rightarrow \alpha
\end{array}
\\]

**Exercise:** Using informal intuitionistic reasoning, prove above.

## Pair type again

Pair type is 

\\[
A \times B = \forall \alpha.(A \rightarrow B \rightarrow \alpha) \rightarrow \alpha
\\]

The conjunction operator is defined as

\\[
A \wedge B \Leftrightarrow \forall \alpha. (A \Rightarrow B \Rightarrow \alpha) \Rightarrow \alpha 
\\]

<center>

<h1 style="text-align:center"> Fin. </h1>
</center>